In [52]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StructType, IntegerType, StringType, FloatType
from pyspark.sql import SQLContext
from pyspark.sql.functions import col
from pyspark.sql.functions import sum, avg

In [53]:
spark = SparkContext(master='local', appName='Dataframes')
sqlContext = SQLContext(spark)

In [54]:
!ls files

deporte.csv	 deportistaError.csv  modelo_relacional.jpg
deportista2.csv  evento.csv	      paises.csv
deportista.csv	 juegos.csv	      resultados.csv


In [55]:
#aca cargo la ruta del archivo
path = "files/"

In [56]:
equiposOlimpicosRDD = spark.textFile(path+'paises.csv') \
    .map(lambda line : line.split(','))

In [87]:
#spark.stop() #stop detiene el contexto

In [58]:
deportistaOlimpicoRDD = spark.textFile(path+'deportista.csv').map(lambda l: l.split(','))
deportistaOlimpicoRDD2 = spark.textFile(path+'deportista2.csv').map(lambda l: l.split(','))

In [59]:
deportistaOlimpicoRDD = deportistaOlimpicoRDD.union(deportistaOlimpicoRDD2)

In [60]:
def eliminaEncabezado(indice, interador):
    return iter(list(interador)[1:])

In [61]:
deportistaOlimpicoRDD = deportistaOlimpicoRDD.mapPartitionsWithIndex(eliminaEncabezado)

In [62]:
deportistaOlimpicoRDD = deportistaOlimpicoRDD.map(lambda l: 
    (
    int(l[0]),
    l[1],
    int(l[2]),
    int(l[3]),
    int(l[4]),
    float(l[5]),
    int(l[6])
    )
)

In [63]:
schema = StructType([
    StructField('deportista_id', IntegerType(), False),
    StructField('nombre', StringType(), False),
    StructField('genero', IntegerType(), False),
    StructField('edad', IntegerType(), False),
    StructField('altura', IntegerType(), False),
    StructField('peso', FloatType(), False),
    StructField('equipo_id', IntegerType(), False )
])

In [64]:
deportistaOlimpicoDF = sqlContext.createDataFrame(deportistaOlimpicoRDD, schema)

In [65]:
evento_schema = StructType([
    StructField('evento_id', IntegerType(), False),
    StructField('evento', StringType(), False),
    StructField('deporte_id', IntegerType(), False)
])

In [66]:
eventoDF = sqlContext.read.format("csv").\
        option("header", True).\
        schema(evento_schema).\
        load(path+"evento.csv")

In [67]:
paises_schema = StructType([
    StructField("equipo_id", IntegerType(), False),
    StructField("equipo", StringType(), False), 
    StructField("sigla", StringType(), False)
])


In [68]:
paisesDF = sqlContext.read.format("csv").\
        option("header", True).\
        schema(paises_schema).\
        load(path+"paises.csv")


In [69]:
resultados_schema = StructType([
    StructField("resultado_id", IntegerType(), False),
    StructField("medalla", StringType(), False),
    StructField("deportista_id", IntegerType(), False),
    StructField("juego_id", IntegerType(), False),
    StructField("evento_id", IntegerType(), False)
])


In [70]:
resultadosDF = sqlContext.read.format("csv").\
            option("header", True).\
            schema(resultados_schema).\
            load(path+"resultados.csv")


In [71]:
deportes_schema = StructType([
    StructField("deporte_id", IntegerType(), False),
    StructField("deporte", StringType(), False)
])

In [72]:
deportesDF = sqlContext.read.format("csv").\
            option("header", True).\
            schema(deportes_schema).\
            load(path+"deporte.csv")

In [73]:
#eliminanado y renombrando columnas
deportistaOlimpicoDF = deportistaOlimpicoDF.withColumnRenamed('genero','sexo').drop('altura')

In [74]:
deportistaOlimpicoDF = deportistaOlimpicoDF.select('deportista_id', 'nombre', col('edad').alias('edadAlJugar'), 'equipo_id')

In [75]:
deportistaOlimpicoDF = deportistaOlimpicoDF.filter(deportistaOlimpicoDF.edadAlJugar != 0)

In [76]:
juego_schema = StructType([
    StructField("juego_id", IntegerType(), False),
    StructField("anio", StringType(), False),
    StructField("temporada", StringType(), False),
    StructField("ciudad", StringType(), False)
])

In [77]:
juegoDF = sqlContext.read.format("csv").\
            option("header", True).\
            schema(juego_schema).\
            load(path+"juegos.csv")

In [79]:
medallistaXAnio = deportistaOlimpicoDF \
    .join(
        resultadosDF, 
        deportistaOlimpicoDF.deportista_id == resultadosDF.deportista_id, 
        "left"
    ) \
    .join(
        juegoDF, 
        juegoDF.juego_id == resultadosDF.juego_id, 
        "left"
    ) \
    .join(
        paisesDF, 
        deportistaOlimpicoDF.equipo_id == paisesDF.equipo_id, 
        "left"
    ) \
    .join(
        eventoDF, 
        eventoDF.evento_id == resultadosDF.evento_id, 
        "left"
    ) \
    .join(
        deportesDF, 
        eventoDF.deporte_id == deportesDF.deporte_id, 
        "left"
    ) \
    .select(
        "sigla",
        "anio",
        "medalla",
        eventoDF.evento.alias("nombre subdisciplina"),
        deportesDF.deporte.alias("nombre disciplina"),
        deportistaOlimpicoDF.nombre    
    )


In [80]:
medallistaXAnio2 = medallistaXAnio.filter(medallistaXAnio.medalla != 'NA') \
    .sort('anio') \
        .groupBy('sigla', 'anio','nombre subdisciplina') \
            .count()

In [81]:
resultadosDF.registerTempTable('resultado')
deportistaOlimpicoDF.registerTempTable('deportista')
paisesDF.registerTempTable('paises')

In [82]:
sqlContext.sql('SELECT * FROM deportista').show(5)

+-------------+--------------------+-----------+---------+
|deportista_id|              nombre|edadAlJugar|equipo_id|
+-------------+--------------------+-----------+---------+
|            1|           A Dijiang|         24|      199|
|            2|            A Lamusi|         23|      199|
|            3| Gunnar Nielsen Aaby|         24|      273|
|            4|Edgar Lindenau Aabye|         34|      278|
|            5|Christine Jacoba ...|         21|      705|
+-------------+--------------------+-----------+---------+
only showing top 5 rows



In [85]:
sqlContext.sql('''SELECT DISTINCT medalla, equipo, sigla
                    FROM resultado r
                    JOIN deportista d
                      ON r.deportista_id = d.deportista_id
                    JOIN paises p
                      ON p.equipo_id = d.equipo_id
                   WHERE medalla <> "NA"
                   ORDER BY sigla DESC
                    ''').show()

+-------+--------------------+-----+
|medalla|              equipo|sigla|
+-------+--------------------+-----+
| Silver|            Zimbabwe|  ZIM|
| Bronze|            Zimbabwe|  ZIM|
|   Gold|            Zimbabwe|  ZIM|
| Silver|              Zambia|  ZAM|
|   Gold|          Yugoslavia|  YUG|
| Silver|          Yugoslavia|  YUG|
| Bronze|          Yugoslavia|  YUG|
| Bronze|West Indies Feder...|  WIF|
| Silver|             Vietnam|  VIE|
|   Gold|             Vietnam|  VIE|
|   Gold|           Venezuela|  VEN|
| Bronze|           Venezuela|  VEN|
| Silver|           Venezuela|  VEN|
| Silver|          Uzbekistan|  UZB|
|   Gold|          Uzbekistan|  UZB|
| Bronze|          Uzbekistan|  UZB|
|   Gold|New York Athletic...|  USA|
| Bronze|New York Athletic...|  USA|
|   Gold|            Minotaur|  USA|
|   Gold|            Kathleen|  USA|
+-------+--------------------+-----+
only showing top 20 rows

